In [2]:
from pygeopkg.core.geopkg import GeoPackage
from pygeopkg.core.srs import SRS
from pygeopkg.core.field import Field
from pygeopkg.shared.enumeration import GeometryType, SQLFieldTypes
import xarray as xr
import numpy as np
from geopandas.tools import sjoin

In [3]:
#create a geopackage
gpkg=GeoPackage.create(r'C:\Users\Sysadmin\Desktop\QGIS\Python codes\linking_layers.gpkg', flavor = 'EPSG')

#Fill in well known text from dataset
data=xr.load_dataarray(r'C:\Users\Sysadmin\Desktop\QGIS\wreck_trajectories.nc')
srs_wkt = data.attrs['wkt']

srs = SRS('Custom', 'EPSG', 4326, srs_wkt)

#fields and feature class of the first trajectory_table aka child_table
fields = (Field('serial_no', SQLFieldTypes.integer),
          Field('start_time', SQLFieldTypes.datetime))
fc = gpkg.create_feature_class ('child_table', srs, fields = fields, shape_type = GeometryType.linestring)

#fields and feature class of the second mission_table aka parent_table
fields2 = (Field('mission_name', SQLFieldTypes.text),
          Field('date', SQLFieldTypes.date),
          Field('vehicle_name', SQLFieldTypes.text),
          Field('serial_no', SQLFieldTypes.integer),
          Field('opening_angle', SQLFieldTypes.integer),
          Field('frequency', SQLFieldTypes.integer),
          Field('image_resolution', SQLFieldTypes.text))
fc2 = gpkg.create_feature_class('parent_table', srs, fields = fields2, shape_type = GeometryType.point)

#fields3 = (Field('serial_no', SQLFieldTypes.integer),
         #  Field('polygon_shape', SQLFieldTypes.text))
#fc3 = gpkg.create_feature_class('mission_table', srs, fields = fields2, shape_type = GeometryType.point)

In [4]:
from pygeopkg.shared.constants import SHAPE
from pygeopkg.conversion.to_geopkg_geom import (points_zm_to_gpkg_line_string_zm,point_to_gpkg_point, make_gpkg_geom_header)
from pygeopkg.core.geopkg import GeoPackage

gpkg = GeoPackage(r'C:\Users\Sysadmin\Desktop\QGIS\Python codes\linking_layers.gpkg')
fc = gpkg.get_feature_class ('child_table')
fc2 = gpkg.get_feature_class ('parent_table')

field_names = [SHAPE, 'serial_no','start_time']
field_names2 = [SHAPE, 'mission_name','date','vehicle_name','serial_no','opening_angle','frequency','image_resolution']

hdr =  make_gpkg_geom_header(fc.srs.srs_id)

#input values into the child_table
rows=[]
for i in range(len(data['leg'])):
    line = data[i].transpose('index', 'xyz')
    shape = list(line.shape)
    shape[-1] = 4
    arr = np.zeros(shape)
    arr[:, 0] = line[:, 1]
    arr[:, 1] = line[:, 0]
    arr[:, 2] = line[:, 2]

    gpkg_wkb = points_zm_to_gpkg_line_string_zm(hdr, arr)

    rows.append([gpkg_wkb,i,'2023-11-14 14:28:00'])

fc.insert_rows(field_names, rows)

#input values into the parent_table
rows2=[]
points = [(-119, 34),(-120,35)]
for x,y in points:
    gpkg_wkb2 = point_to_gpkg_point(hdr,x,y)

    rows2.append([gpkg_wkb2,'Mission Zeus','2023-11-14','Vehicle Zeus','1','45','100','480p'])

fc2.insert_rows(field_names2, rows2)


In [6]:
import geopandas
example = geopandas.read_file(r'C:\Users\Sysadmin\Desktop\QGIS\Python codes\linking_layers.gpkg') #Returns a GeoDataFrame object
example

,serial_no,start_time,geometry
0,0,2023-11-14 14:28:00,"LINESTRING Z (9.56169 44.18651 -7.80227, 9.561..."
1,1,2023-11-14 14:28:00,"LINESTRING Z (9.55664 44.18917 -11.87498, 9.55..."
2,2,2023-11-14 14:28:00,"LINESTRING Z (9.56161 44.18923 -15.02603, 9.56..."
3,3,2023-11-14 14:28:00,"LINESTRING Z (9.56132 44.18941 -14.46273, 9.56..."
4,4,2023-11-14 14:28:00,"LINESTRING Z (9.55676 44.18637 -5.84171, 9.557..."
5,5,2023-11-14 14:28:00,"LINESTRING Z (9.55696 44.18931 -11.97353, 9.55..."
6,6,2023-11-14 14:28:00,"LINESTRING Z (9.56221 44.18682 -8.93474, 9.561..."
7,7,2023-11-14 14:28:00,"LINESTRING Z (9.55761 44.18579 -4.46144, 9.558..."
8,8,2023-11-14 14:28:00,"LINESTRING Z (9.55639 44.18895 -12.02914, 9.55..."
9,9,2023-11-14 14:28:00,"LINESTRING Z (9.55602 44.18886 -11.93023, 9.55..."


In [7]:
example1 = geopandas.read_file(
    r'C:\Users\Sysadmin\Desktop\QGIS\Python codes\linking_layers.gpkg', 
    layer ='parent_table', 
    #rows=3, 
    #include_fields=["serial_no","mission_name"],
    ignore_geometry=False, 
)
example1

,mission_name,date,vehicle_name,serial_no,opening_angle,frequency,image_resolution,geometry
0,Mission Zeus,2023-11-14,Vehicle Zeus,1,45,100,480p,POINT (-119.00000 34.00000)
1,Mission Zeus,2023-11-14,Vehicle Zeus,1,45,100,480p,POINT (-120.00000 35.00000)
